# Family of Wavelets

- A prototype wavelet (canonically a [Mother wavelet](https://en.wikipedia.org/wiki/Wavelet#Mother_wavelet)), which defines the overall mathematical struture for the entire wavelet family. We refer to wavelet and kernels synonymously here.
- A family is comprised of the Mother wavelet and its children -- a set of scaled and shifted wavelets derived from the Mother.
- Wavelet children are essentially feature selectors that are sensitive to fluctuations of different multi-dimensional scales.

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from pyeisen import family

## Working with the Morlet Wavelet

A [Morlet Wavelet](https://en.wikipedia.org/wiki/Morlet_wavelet) (${\displaystyle \psi}$) is one type of mother wavelet that is sensitive to time-limited and band-limited sinusoidal fluctuations. Conceptually, the mother wavlet of the Morlet family is generated by multiplying a sinuisoidal wave of a single frequency with a time-varying Gaussian envelope. Children are based on parametrizations of these components:

  ${\displaystyle \psi _{\omega,\sigma}(t) = C * e^{j{\omega}t} * e^{\frac{-t^{2}}{\sigma^2}}}$

- The sinusoid ($e^{j{\omega}t}$) is parametrized by:
  - Center or fundamental frequency ($\omega$) of the child wavelet.
    - **Note:** this "exponential representation" is derived from [Euler's Formula](https://en.wikipedia.org/wiki/Euler's_formula), which is a mathematical tool for representing cicular data in the complex-valued domain:
          ${\displaystyle Ce^{j{\omega}t}=C\cos {\omega}t + jC \sin {\omega}t}$;  where $j = \sqrt{-1}$
- The time-varying Gaussian envelope  ($e^{\frac{-t^{2}}{\sigma^2}}$) is parametrized by:
    -  Standard deviation ($\sigma$) defines the width or roll-off of the masking envelope.
    -  Larger standard deviation results in a Gaussian envelope with a slower and wider roll-off. The result of increasing the standard deviation is that the child wavelet becomes longer and samples more cycles/periods of the sinusoid. This yields a more precise measurement of the frequency content of the signal (reduces the bandwidth around the center frequency), but trades off temporal precision. In contrast, a smaller standard deviation improves temporal precision by shortening the length of the wavelet but reduces frequency precision (increases spectral leakage by increasing bandwidth around center frequency).
    -  Standard deviation dictates the length of the child wavelet by directly constraining the number of cycles ($n$), or periods, of the sinusoid that are measured ($\sigma = \frac{n}{\omega}$).
- A normalization constant $C$ is also defined to ensure that different child wavelets are weighted equally. Put another way, the child wavelets must have the same ["energy"](https://en.wikipedia.org/wiki/Energy_(signal_processing)), constrained as follows:

  $\sum{|\psi _{\omega,\sigma}(t)^2|} = 1$
  
  Therefore, $C = \frac{1}{\sum{|\psi _{\omega,\sigma}(t)^2|}}$.

### Mechanics of the Morlet wavelet using a single child (kernel) example

- Let's generate a single child wavelet with a center frequency of `5 Hz (cycles/sec)` and `5 cycles`.
- Based on these specifications, we can expect that the kernel will be approximately `1 sec` in duration.

In [ ]:
# Generate a wavelet family based on parameters
morlet_fam = family.morlet(
    freqs=np.array([5.0]),       # Center frequencies of the children (Cycles/sec (Hz))
    cycles=np.array([5]),        # Number of cycles to sample for each child. Governs the width of the Gaussian envelope.
    fs=1000.0,                   # Sampling frequency of the wavelet (depends on the sampling frequency of the signal to be analyzed); set arbitrarily large here.
    n_win=6.5                      # Window support size for the wavelet -- wide enough to sample enough of the envelope roll-off.
)

print('Wavelet kernel is defined by a set of complex-valued weights:\n')
display(morlet_fam['kernel'][0, ::100])

- In the complex-representation below note the major features of the child wavelet.
    - Each of the two components is comprised of 5 periods within the 1 second span of the wavelet length. This number of periods is based on the `cycles` parameter defined in the wavelet construction code.
    - The presence of 5 cycles over the 1 second wavelet length indicates that the center frequency of the wavelet is 5 Hz.
    - Magnitudes of both components decay as you move away from the center of the window due to the Gaussian-based enveloping of the sinusoid. This feature is essential to the time-frequency feature provided by the wavelet. This point is illustrated further in the example that follows.
    - Real and imaginary components are time-shifted versions of each other. This time-shift helps the wavelet to discern the instantaneous phase (phase at a discrete point in time) of the sinusoid. Example below illustrates this further:

In [ ]:
# visualize a complex-valued representation of the wavelet
plt.plot(morlet_fam['sample']['time'], morlet_fam['kernel'].real.T, label='Real Component')
plt.plot(morlet_fam['sample']['time'], morlet_fam['kernel'].imag.T, label='Imaginary Component')
plt.ylabel('Kernel Weight (a.u.)')
plt.xlabel('Kernel Time Window (s)')
plt.legend()
plt.title('Complex Representation')
plt.show()

- In the Euler's representation below note the major features of the child wavelet.
    - Amplitude indicates where the energy of the wavelet is concentrated in time. Energy decays as you move away from the center of the window -- this suggests that the wavelet is most sensitive to sinusoidal fluctuations at the center of the window. This is critical for localizing the occurrence of the sinusoid in time.
    - Phase indicates the location (angle) of the sinusoid in circular space. The angle will shift from $-\pi$ to $\pi$ over a single cycle of the sinusoid.
    - Using the phase plot, notice that the wavelet defines approximately 5 full cycles of the sinusoid within the window.
    - Combination of instantaneous amplitude and phase at a given point within the wavelet time window indicates the precision.

In [ ]:
# Alternatively, we can visualize the Euler's representation of the wavelet
# based on the amplitude/envelope component and the sinusoidal phase component

plt.figure()
ax = plt.subplot(111)
color = 'tab:blue'
ax.plot(morlet_fam['sample']['time'], np.abs(morlet_fam['kernel']).T, color=color)
ax.set_ylabel('Wavelet Amplitude', color=color)
ax.tick_params(axis='y', labelcolor=color)


ax2 = ax.twinx() 
color = 'tab:red'
ax2.plot(morlet_fam['sample']['time'], np.angle(morlet_fam['kernel']).T, color=color)
ax2.set_ylabel('Wavelet Phase', color=color)
ax2.set_yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
ax2.set_yticklabels(['$-\pi$', '$-\pi/2$', '$0$', '$\pi/2$', '$\pi$']) #, '$-\frac{\pi}{2}$', '$0$', '$\frac{\pi}{2}$', '$\frac{\pi}$'])
ax2.tick_params(axis='y', labelcolor=color)

ax.set_xlabel('Kernel Time Window (s)')
ax.set_title('Euler\'s Representation')
plt.show()

- Let's modify the single child wavelet using the same center frequency of `5 Hz (cycles/sec)` but increasing the number of cycles to `12 cycles`.
- Based on these specifications, we can expect that the kernel will be approximately `2.4 sec` in duration.

In [ ]:
# Generate a wavelet family based on parameters
morlet_fam_2 = family.morlet(
    freqs=np.array([5.0]),       # Center frequencies of the children (Cycles/sec (Hz))
    cycles=np.array([12]),        # Number of cycles to sample for each child. Governs the width of the Gaussian envelope.
    fs=1000.0,                   # Sampling frequency of the wavelet (depends on the sampling frequency of the signal to be analyzed); set arbitrarily large here.
    n_win=6.5                      # Window support size for the wavelet -- wide enough to sample enough of the envelope roll-off.
)

- Notice now that the child wavelet defined a sinusoid with a frequency of 5 Hz and spanning a time length of approximately 12 cycles.
- The overall duration of the kernel has increased from before, and the roll-off of the amplitude occurs more gradually over the span of the 1 second centered window from before.
- The new child wavelet has stricter requirements of the length of the sinusoidal pattern it is sensitive to, but it is will be more imprecise in quantifying the time course of the sinusoid.

In [ ]:
# Alternatively, we can visualize the Euler's representation of the wavelet
# based on the amplitude/envelope component and the sinusoidal phase component

plt.figure()
ax = plt.subplot(111)
color = 'tab:blue'
ax.plot(morlet_fam_2['sample']['time'], np.abs(morlet_fam_2['kernel']).T, color=color)
ax.set_ylabel('Wavelet Amplitude', color=color)
ax.tick_params(axis='y', labelcolor=color)


ax2 = ax.twinx() 
color = 'tab:red'
ax2.plot(morlet_fam_2['sample']['time'], np.angle(morlet_fam_2['kernel']).T, color=color)
ax2.set_ylabel('Wavelet Phase', color=color)
ax2.set_yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
ax2.set_yticklabels(['$-\pi$', '$-\pi/2$', '$0$', '$\pi/2$', '$\pi$']) #, '$-\frac{\pi}{2}$', '$0$', '$\frac{\pi}{2}$', '$\frac{\pi}$'])
ax2.tick_params(axis='y', labelcolor=color)

ax.set_xlabel('Kernel Time Window (s)')
ax.set_title('Euler\'s Representation')
plt.show()